In [7]:
import pyspark # only run after findspark.init()
from pyspark import SparkContext, SparkConf
conf = SparkConf()
sc = SparkContext(conf=conf)


ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-1-76dfea3d5006>:4 

In [10]:
from pyspark.sql import SQLContext
sqlContext=SQLContext(sc)
from pyspark.sql.types import IntegerType
import pandas as pd
from pyspark.sql.functions import udf
df_pd = pd.DataFrame(
    data={'integers': [1, 2, 3],
     'floats': [-1.0, 0.5, 2.7],
     'integer_arrays': [[1, 2], [3, 4, 5], [6, 7, 8, 9]]}
)
df = sqlContext.createDataFrame(df_pd)
def square(x):
    return x**2

square_udf_int = udf(lambda z: square(z), IntegerType())
(df.select('integers',
              'floats',
              square_udf_int('integers').alias('int_squared'),
              square_udf_int('floats').alias('float_squared'))
    .show())

+--------+------+-----------+-------------+
|integers|floats|int_squared|float_squared|
+--------+------+-----------+-------------+
|       1|  -1.0|          1|         null|
|       2|   0.5|          4|         null|
|       3|   2.7|          9|         null|
+--------+------+-----------+-------------+



In [13]:
data = [('Foo',10,'US',3),\
                ('Foo',39,'UK',1),\
                ('Bar',57,'IN',2),\
                ('Bar',72,'CA',2),\
                ('Baz',22,'US',6),\
                ('Baz',23,'UK',6)]
df = sqlContext.createDataFrame(data,['name','age','country','id'])

In [12]:
from pyspark.sql.types import StringType
sqlContext.registerFunction(
                "RankToFlag", lambda x: "Y" if x > 1 else "N", StringType())

<function __main__.<lambda>(x)>

In [16]:
keyCols = 'country'
orderStr = 'id'
queryStr = 'select *, RankToFlag(ROW_NUMBER() over (partition by ' \
                       + keyCols + ' order by ' + orderStr + ' )) as rank ' \
                                                             ' from df order by ' + orderStr

In [28]:
df.withColumnRenamed('name', 'firstname').show()

+---------+---+-------+---+
|firstname|age|country| id|
+---------+---+-------+---+
|      Foo| 10|     US|  3|
|      Foo| 39|     UK|  1|
|      Bar| 57|     IN|  2|
|      Bar| 72|     CA|  2|
|      Baz| 22|     US|  6|
|      Baz| 23|     UK|  6|
+---------+---+-------+---+



In [26]:
df.filter(df.name == 'Foo').drop('age').show()

+----+-------+---+
|name|country| id|
+----+-------+---+
| Foo|     US|  3|
| Foo|     UK|  1|
+----+-------+---+



In [37]:
df2 = df.dropDuplicates(subset=['name'])

In [41]:
df.join(df2,df.name==df2.name,'inner').show()

+----+---+-------+---+----+---+-------+---+
|name|age|country| id|name|age|country| id|
+----+---+-------+---+----+---+-------+---+
| Bar| 57|     IN|  2| Bar| 57|     IN|  2|
| Bar| 72|     CA|  2| Bar| 57|     IN|  2|
| Foo| 10|     US|  3| Foo| 10|     US|  3|
| Foo| 39|     UK|  1| Foo| 10|     US|  3|
| Baz| 22|     US|  6| Baz| 22|     US|  6|
| Baz| 23|     UK|  6| Baz| 22|     US|  6|
+----+---+-------+---+----+---+-------+---+



In [43]:
df.printSchema()dtypes

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- country: string (nullable = true)
 |-- id: long (nullable = true)



In [50]:
df.select('name','age').show()

+----+---+
|name|age|
+----+---+
| Foo| 10|
| Foo| 39|
| Bar| 57|
| Bar| 72|
| Baz| 22|
| Baz| 23|
+----+---+



In [52]:
df.where('country' + "='IN'").show()

+----+---+-------+---+
|name|age|country| id|
+----+---+-------+---+
| Bar| 57|     IN|  2|
+----+---+-------+---+



In [ ]:
df.withColumnRenamed('name', 'firstname').show()
df.filter(df.name == 'Foo').drop('age').drop('country').show()
df2 = df.dropDuplicates(subset=['name'])
df.join(df2,df.name==df2.name,'inner').show()
df.where('country '+ "='IN'").show()
df.select('name','age').show()

In [54]:
df.sort('name','id').show()
#dff = sqlContext.sql(query).fillna('')

+----+---+-------+---+
|name|age|country| id|
+----+---+-------+---+
| Bar| 72|     CA|  2|
| Bar| 57|     IN|  2|
| Baz| 23|     UK|  6|
| Baz| 22|     US|  6|
| Foo| 39|     UK|  1|
| Foo| 10|     US|  3|
+----+---+-------+---+



In [56]:
from pyspark.sql.functions import udf, lit, monotonically_increasing_id
df.withColumn("ROW_NUM", monotonically_increasing_id()).show()

+----+---+-------+---+-----------+
|name|age|country| id|    ROW_NUM|
+----+---+-------+---+-----------+
| Foo| 10|     US|  3| 8589934592|
| Foo| 39|     UK|  1|17179869184|
| Bar| 57|     IN|  2|25769803776|
| Bar| 72|     CA|  2|42949672960|
| Baz| 22|     US|  6|51539607552|
| Baz| 23|     UK|  6|60129542144|
+----+---+-------+---+-----------+



In [ ]:
#collease vs repartition
Node 1 = 1,2,3
Node 2 = 4,5,6
Node 3 = 7,8,9
Node 4 = 10,11,12

Then coalesce down to 2 partitions:

Node 1 = 1,2,3 + (10,11,12)
Node 3 = 7,8,9 + (4,5,6)
coalesce uses existing partitions to minimize the amount of data 
that's shuffled.  repartition creates new partitions and does a full shuffle
coalesce results in partitions with different amounts of data 
(sometimes partitions that have much different sizes) and repartition results in roughly equal sized partitions.

coalesce may run faster than repartition, but unequal sized partitions are generally slower to work with 
than equal sized partitions. You ll usually need to repartition datasets after filtering a large data set
. I ve found repartition to be faster overall because Spark is built to work with equal sized partitions.

repartition - its recommended to use repartition while increasing no of partitions, 
because it involve shuffling of all the data.

coalesce- it’s is recommended to use coalesce while reducing no of partitions

In [ ]:
#persistance
With cache(), you use only the default storage level MEMORY_ONLY. With persist(), you can specify which storage level you want

MEMORY_ONLY--deserialized,recompute them next time whenever needed
MEMORY_ONLY_SER--serialized,space efficient as compared to deserialized objects,
MEMORY_AND_DISK--stores the excess partition on the disk,
MEMORY_AND_DISK_SER
DISK_ONLY


In [ ]:
--executor mem=50G
--num-exe =10
--exe-core =4
5 tasks per executor --  . So it’s good to keep the number of cores per executor below that number

In [ ]:
#word count prob
counts = lines.flatMap(lambda x: x.split(' ')) \
                  .map(lambda x: (x, 1)) \
                  .reduceByKey(add)
    output = counts.collect()
    for (word, count) in output:
        print("%s: %i" % (word, count))

In [63]:
a=['a','b']
b=['c','d','e']
c=a+b

In [64]:
c

['a', 'b', 'c', 'd', 'e']

In [65]:
a=[1,2,3,4,1]
z=a[0]
for i in a:
    if i>z:
        z=i
print(z)        

4


In [67]:
import pyspark.sql.functions as F
df.withColumn(
            "no_basket",
            F.when(
                F.col("id") != 0,
                0
            ).otherwise(
                1
            )).show()

+----+---+-------+---+---------+
|name|age|country| id|no_basket|
+----+---+-------+---+---------+
| Foo| 10|     US|  3|        0|
| Foo| 39|     UK|  1|        0|
| Bar| 57|     IN|  2|        0|
| Bar| 72|     CA|  2|        0|
| Baz| 22|     US|  6|        0|
| Baz| 23|     UK|  6|        0|
+----+---+-------+---+---------+



In [74]:
df.filter(
    F.col('age')>50
).select('name','country').show()

+----+-------+
|name|country|
+----+-------+
| Bar|     IN|
| Bar|     CA|
+----+-------+



In [76]:
df.groupBy(df.name).agg(F.sum('age')).show()

Py4JJavaError: An error occurred while calling o480.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 97.0 failed 1 times, most recent failure: Lost task 0.0 in stage 97.0 (TID 1000, localhost, executor driver): java.io.FileNotFoundException: C:\Users\vprasad\AppData\Local\Temp\blockmgr-746bdcf0-d3b5-4c40-a270-266f0322557d\39\shuffle_9_0_0.index.a1570f6e-7cf2-4324-a710-0b72bd8a47d4 (The system cannot find the path specified)
	at java.io.FileOutputStream.open0(Native Method)
	at java.io.FileOutputStream.open(Unknown Source)
	at java.io.FileOutputStream.<init>(Unknown Source)
	at java.io.FileOutputStream.<init>(Unknown Source)
	at org.apache.spark.shuffle.IndexShuffleBlockResolver.writeIndexFileAndCommit(IndexShuffleBlockResolver.scala:144)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSortShuffleWriter.java:127)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:96)
	at org.a

In [80]:
from pyspark.sql.window import Window
df.withColumn("min age",F.min('age').over(Window.partitionBy('name'))).show()

Py4JJavaError: An error occurred while calling o563.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 99.0 failed 1 times, most recent failure: Lost task 3.0 in stage 99.0 (TID 1011, localhost, executor driver): java.io.FileNotFoundException: C:\Users\vprasad\AppData\Local\Temp\blockmgr-746bdcf0-d3b5-4c40-a270-266f0322557d\04\temp_shuffle_e79bde69-9144-432f-be80-c600be2846c1 (The system cannot find the path specified)
	at java.io.FileOutputStream.open0(Native Method)
	at java.io.FileOutputStream.open(Unknown Source)
	at java.io.FileOutputStream.<init>(Unknown Source)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:103)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:116)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:237)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSortShuffleWriter.